## Event based

In [1]:
import climtas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xarray as xr

In [2]:
# opening the input files with heatwave severity data 
thw = xr.open_dataset('/g/data/e14/cp3790/Charuni/Tasmania/aus-coastal-sev.nc')
mhw = xr.open_dataset('/g/data/e14/cp3790/Charuni/Tasmania/aus-ocean-sev-2.nc')

In [3]:
# using climtas.event.find_events to identify days where severity>1 for a minimum of 3 consecutive days 
thw_events = climtas.event.find_events(thw.severity > 1, min_duration = 3)
thw_events

,time,latitude,longitude,event_duration
0,0,48,151,3
1,0,48,152,3
2,0,50,152,3
3,0,54,154,3
4,0,54,155,3
...,...,...,...,...
165334,13508,130,142,6
165335,13508,130,143,6
165336,13508,131,142,6
165337,13508,133,140,6


In [4]:
def get_coords(da, events):
    # Convert the index values to coordinates
    coords = {}
    for d in da.dims:
        coords[d] = da[d].values[events[d].values]
    
    # Also work out when the event ends
    coords['time_end'] = da['time'].values[events['time'].values + events['event_duration'].values-1]
    #coords['event_duration'] = coords['time_end'] - coords['time'] 
    coords['event_duration'] = events['event_duration'].values
    
    return pd.DataFrame(coords, index=events.index)

In [5]:
thw_new = get_coords(thw, thw_events)
thw_new

,latitude,longitude,time,time_end,event_duration
0,-22.00,150.75,1982-01-01,1982-01-03,3
1,-22.00,151.00,1982-01-01,1982-01-03,3
2,-22.50,151.00,1982-01-01,1982-01-03,3
3,-23.50,151.50,1982-01-01,1982-01-03,3
4,-23.50,151.75,1982-01-01,1982-01-03,3
...,...,...,...,...,...
165334,-42.50,148.50,2018-12-26,2018-12-31,6
165335,-42.50,148.75,2018-12-26,2018-12-31,6
165336,-42.75,148.50,2018-12-26,2018-12-31,6
165337,-43.25,148.00,2018-12-26,2018-12-31,6


In [6]:
thw_new_df = thw_new.reset_index()
thw_new_df.set_index(['latitude', 'longitude', 'time'], inplace=True)
thw_new_df

index   time_end  event_duration
latitude longitude time                                         
-22.00   150.75    1982-01-01       0 1982-01-03               3
         151.00    1982-01-01       1 1982-01-03               3
-22.50   151.00    1982-01-01       2 1982-01-03               3
-23.50   151.50    1982-01-01       3 1982-01-03               3
         151.75    1982-01-01       4 1982-01-03               3
...                               ...        ...             ...
-42.50   148.50    2018-12-26  165334 2018-12-31               6
         148.75    2018-12-26  165335 2018-12-31               6
-42.75   148.50    2018-12-26  165336 2018-12-31               6
-43.25   148.00    2018-12-26  165337 2018-12-31               6
         148.25    2018-12-26  165338 2018-12-31               6

[165339 rows x 3 columns]

In [7]:
# using climtas.event.find_events to identify days where severity>1 for a minimum of 5 consecutive days (MHW events)
mhw_events = climtas.event.find_events(mhw.severity > 1, min_duration = 5)
mhw_events

,time,latitude,longitude,event_duration
0,0,0,29,5
1,0,1,123,5
2,0,1,124,5
3,0,4,94,5
4,0,4,95,5
...,...,...,...,...
127727,13497,135,133,17
127728,13500,135,134,14
127729,13500,135,135,14
127730,13491,135,136,23


In [8]:
mhw_new = get_coords(mhw, mhw_events)
mhw_new

,latitude,longitude,time,time_end,event_duration
0,-10.00,120.25,1982-01-01,1982-01-05,5
1,-10.25,143.75,1982-01-01,1982-01-05,5
2,-10.25,144.00,1982-01-01,1982-01-05,5
3,-11.00,136.50,1982-01-01,1982-01-05,5
4,-11.00,136.75,1982-01-01,1982-01-05,5
...,...,...,...,...,...
127727,-43.75,146.25,2018-12-15,2018-12-31,17
127728,-43.75,146.50,2018-12-18,2018-12-31,14
127729,-43.75,146.75,2018-12-18,2018-12-31,14
127730,-43.75,147.00,2018-12-09,2018-12-31,23


In [9]:
mhw_new_df = mhw_new.reset_index()
mhw_new_df.set_index(['latitude', 'longitude', 'time'], inplace=True)
mhw_new_df

index   time_end  event_duration
latitude longitude time                                         
-10.00   120.25    1982-01-01       0 1982-01-05               5
-10.25   143.75    1982-01-01       1 1982-01-05               5
         144.00    1982-01-01       2 1982-01-05               5
-11.00   136.50    1982-01-01       3 1982-01-05               5
         136.75    1982-01-01       4 1982-01-05               5
...                               ...        ...             ...
-43.75   146.25    2018-12-15  127727 2018-12-31              17
         146.50    2018-12-18  127728 2018-12-31              14
         146.75    2018-12-18  127729 2018-12-31              14
         147.00    2018-12-09  127730 2018-12-31              23
         147.25    2018-12-13  127731 2018-12-31              19

[127732 rows x 3 columns]

In [10]:
# merging the THW and MHW dataframes based on latitude and longitude 
df_merge_col = pd.merge(thw_new_df.head(1300), mhw_new_df.head(1300), on=['latitude', 'longitude'])
df_merge_col

index_x time_end_x  event_duration_x  index_y time_end_y  \
latitude longitude                                                             
-22.00   150.75           0 1982-01-03                 3      149 1982-01-08   
         150.75           0 1982-01-03                 3      557 1982-01-26   
         150.75           0 1982-01-03                 3     1008 1982-03-12   
         150.75           0 1982-01-03                 3     1291 1982-03-21   
         150.75         570 1982-02-21                 5      149 1982-01-08   
...                     ...        ...               ...      ...        ...   
-34.75   135.50        1243 1982-08-08                 3      814 1982-02-20   
         135.50        1243 1982-08-08                 3     1105 1982-03-16   
-21.50   114.25        1244 1982-08-09                 3     1257 1982-03-20   
-43.50   147.25        1273 1982-08-09                 3      518 1982-01-23   
-43.75   147.25        1277 1982-08-09                 3      522 1982-01-23   

                    event_duration_y  
latitude longitude                    
-22.00   150.75                    7  
         150.75                    9  
         150.75                    9  
         150.75                    6  
         150.75                    7  
...                              ...  
-34.75   135.50                   11  
         135.50                    7  
-21.50   114.25                    5  
-43.50   147.25                    6  
-43.75   147.25                    6  

[2166 rows x 6 columns]

In [11]:
del df_merge_col['index_y']
del df_merge_col['index_x']
df_merge_col

time_end_x  event_duration_x time_end_y  event_duration_y
latitude longitude                                                          
-22.00   150.75    1982-01-03                 3 1982-01-08                 7
         150.75    1982-01-03                 3 1982-01-26                 9
         150.75    1982-01-03                 3 1982-03-12                 9
         150.75    1982-01-03                 3 1982-03-21                 6
         150.75    1982-02-21                 5 1982-01-08                 7
...                       ...               ...        ...               ...
-34.75   135.50    1982-08-08                 3 1982-02-20                11
         135.50    1982-08-08                 3 1982-03-16                 7
-21.50   114.25    1982-08-09                 3 1982-03-20                 5
-43.50   147.25    1982-08-09                 3 1982-01-23                 6
-43.75   147.25    1982-08-09                 3 1982-01-23                 6

[2166 rows x 4 columns]

In [12]:
# Calculating start dates based on end dates and duration
df_merge_col['time_x'] = df_merge_col['time_end_x'].values - pd.to_timedelta(df_merge_col['event_duration_x'].values-1, unit='d')
df_merge_col['time_y'] = df_merge_col['time_end_y'].values - pd.to_timedelta(df_merge_col['event_duration_y'].values-1, unit='d')

In [13]:
df_merge_col

time_end_x  event_duration_x time_end_y  event_duration_y  \
latitude longitude                                                             
-22.00   150.75    1982-01-03                 3 1982-01-08                 7   
         150.75    1982-01-03                 3 1982-01-26                 9   
         150.75    1982-01-03                 3 1982-03-12                 9   
         150.75    1982-01-03                 3 1982-03-21                 6   
         150.75    1982-02-21                 5 1982-01-08                 7   
...                       ...               ...        ...               ...   
-34.75   135.50    1982-08-08                 3 1982-02-20                11   
         135.50    1982-08-08                 3 1982-03-16                 7   
-21.50   114.25    1982-08-09                 3 1982-03-20                 5   
-43.50   147.25    1982-08-09                 3 1982-01-23                 6   
-43.75   147.25    1982-08-09                 3 1982-01-23                 6   

                       time_x     time_y  
latitude longitude                        
-22.00   150.75    1982-01-01 1982-01-02  
         150.75    1982-01-01 1982-01-18  
         150.75    1982-01-01 1982-03-04  
         150.75    1982-01-01 1982-03-16  
         150.75    1982-02-17 1982-01-02  
...                       ...        ...  
-34.75   135.50    1982-08-06 1982-02-10  
         135.50    1982-08-06 1982-03-10  
-21.50   114.25    1982-08-07 1982-03-16  
-43.50   147.25    1982-08-07 1982-01-18  
-43.75   147.25    1982-08-07 1982-01-18  

[2166 rows x 6 columns]

In [20]:
#Calculating the overlap between THW and MHW events 
from datetime import datetime
from collections import namedtuple
Range = namedtuple('Range', ['start', 'end'])
overlap = []
for n in range (df_merge_col.shape[0]):
    mhw = Range(start=df_merge_col['time_y'][n], end=df_merge_col['time_end_y'][n])
    thw = Range(start=df_merge_col['time_x'][n], end=df_merge_col['time_end_x'][n])
    latest_start = max(mhw.start, thw.start)
    earliest_end = min(mhw.end, thw.end)
    delta = (earliest_end - latest_start).days + 1
    b = max(0, delta)
    overlap.append(b)

In [49]:
#Retaining only the THW events which had an overlap of at least one day with a MHW event; these are the co-occurring events
mod_fd = df_merge_col.assign(overlap_days = overlap)
mod_fd = mod_fd[mod_fd.overlap_days != 0]
mod_fd

time_end_x  event_duration_x time_end_y  event_duration_y  \
latitude longitude                                                             
-22.00   150.75    1982-01-03                 3 1982-01-08                 7   
         150.75    1982-03-08                 6 1982-03-12                 9   
         150.75    1982-03-14                 3 1982-03-12                 9   
         151.00    1982-01-03                 3 1982-01-08                 7   
         151.00    1982-03-08                 6 1982-03-20                17   
...                       ...               ...        ...               ...   
-14.00   144.50    1982-03-13                 7 1982-03-15                 9   
-14.25   143.75    1982-03-13                 5 1982-03-15                 8   
-14.50   145.50    1982-03-13                 9 1982-03-18                13   
-13.00   143.50    1982-03-14                 6 1982-03-11                 5   
-14.00   137.00    1982-03-16                 3 1982-03-15                 7   

                       time_x     time_y  overlap_days  
latitude longitude                                      
-22.00   150.75    1982-01-01 1982-01-02             2  
         150.75    1982-03-03 1982-03-04             5  
         150.75    1982-03-12 1982-03-04             1  
         151.00    1982-01-01 1982-01-02             2  
         151.00    1982-03-03 1982-03-04             5  
...                       ...        ...           ...  
-14.00   144.50    1982-03-07 1982-03-07             7  
-14.25   143.75    1982-03-09 1982-03-08             5  
-14.50   145.50    1982-03-05 1982-03-06             8  
-13.00   143.50    1982-03-09 1982-03-07             3  
-14.00   137.00    1982-03-14 1982-03-09             2  

[397 rows x 7 columns]

In [22]:
#Where there is no overlap between events, these are the stand alone THW events
mod_fd = df_merge_col.assign(overlap_days = overlap)
thw_only = mod_fd[mod_fd.overlap_days == 0]
thw_only

time_end_x  event_duration_x time_end_y  event_duration_y  \
latitude longitude                                                             
-22.00   150.75    1982-01-03                 3 1982-01-26                 9   
         150.75    1982-01-03                 3 1982-03-12                 9   
         150.75    1982-01-03                 3 1982-03-21                 6   
         150.75    1982-02-21                 5 1982-01-08                 7   
         150.75    1982-02-21                 5 1982-01-26                 9   
...                       ...               ...        ...               ...   
-34.75   135.50    1982-08-08                 3 1982-02-20                11   
         135.50    1982-08-08                 3 1982-03-16                 7   
-21.50   114.25    1982-08-09                 3 1982-03-20                 5   
-43.50   147.25    1982-08-09                 3 1982-01-23                 6   
-43.75   147.25    1982-08-09                 3 1982-01-23                 6   

                       time_x     time_y  overlap_days  
latitude longitude                                      
-22.00   150.75    1982-01-01 1982-01-18             0  
         150.75    1982-01-01 1982-03-04             0  
         150.75    1982-01-01 1982-03-16             0  
         150.75    1982-02-17 1982-01-02             0  
         150.75    1982-02-17 1982-01-18             0  
...                       ...        ...           ...  
-34.75   135.50    1982-08-06 1982-02-10             0  
         135.50    1982-08-06 1982-03-10             0  
-21.50   114.25    1982-08-07 1982-03-16             0  
-43.50   147.25    1982-08-07 1982-01-18             0  
-43.75   147.25    1982-08-07 1982-01-18             0  

[1769 rows x 7 columns]

In [60]:
new_df = mod_fd[mod_fd.overlap_days != 0].reset_index()
new_df

,latitude,longitude,time_end_x,event_duration_x,time_end_y,event_duration_y,time_x,time_y,overlap_days
0,-22.00,150.75,1982-01-03,3,1982-01-08,7,1982-01-01,1982-01-02,2
1,-22.00,150.75,1982-03-08,6,1982-03-12,9,1982-03-03,1982-03-04,5
2,-22.00,150.75,1982-03-14,3,1982-03-12,9,1982-03-12,1982-03-04,1
3,-22.00,151.00,1982-01-03,3,1982-01-08,7,1982-01-01,1982-01-02,2
4,-22.00,151.00,1982-03-08,6,1982-03-20,17,1982-03-03,1982-03-04,5
...,...,...,...,...,...,...,...,...,...
392,-14.00,144.50,1982-03-13,7,1982-03-15,9,1982-03-07,1982-03-07,7
393,-14.25,143.75,1982-03-13,5,1982-03-15,8,1982-03-09,1982-03-08,5
394,-14.50,145.50,1982-03-13,9,1982-03-18,13,1982-03-05,1982-03-06,8
395,-13.00,143.50,1982-03-14,6,1982-03-11,5,1982-03-09,1982-03-07,3


In [61]:
new_df.to_xarray()

<xarray.Dataset>
Dimensions:           (index: 397)
Coordinates:
  * index             (index) int64 0 1 2 3 4 5 6 ... 391 392 393 394 395 396
Data variables:
    latitude          (index) float64 -22.0 -22.0 -22.0 ... -14.5 -13.0 -14.0
    longitude         (index) float64 150.8 150.8 150.8 ... 145.5 143.5 137.0
    time_end_x        (index) datetime64[ns] 1982-01-03 ... 1982-03-16
    event_duration_x  (index) int64 3 6 3 3 6 3 3 4 6 3 ... 3 4 5 4 4 7 5 9 6 3
    time_end_y        (index) datetime64[ns] 1982-01-08 ... 1982-03-15
    event_duration_y  (index) int64 7 9 9 7 17 17 7 7 9 ... 13 13 13 9 8 13 5 7
    time_x            (index) datetime64[ns] 1982-01-01 ... 1982-03-14
    time_y            (index) datetime64[ns] 1982-01-02 ... 1982-03-09
    overlap_days      (index) int64 2 5 1 2 5 3 1 4 5 1 ... 3 4 5 4 4 7 5 8 3 2